In [ ]:
import pandas as pd
from sklearn import svm
import numpy as np
import shap
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from keras.optimizers import Adam
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import lightgbm as lgb
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import ExtraTreesClassifier
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm

In [ ]:
import sys
sys.path.append('..')
sys.path.append('../../')
from util import Transform, PredictWay, get_k_fold_data, dnn
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import auc
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['Microsoft YaHei']
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import permutation_test_score
from metric import getMATRIX

In [ ]:
'''
preturn
'''

MATRIX = []
def str2Nan(x):
    try:
        return float(x)
    except:
        return np.nan
seed=4 
patient = pd.read_excel('../../)
# put into the data


transform = Transform(seed_num=seed)
patient = transform.confuse(patient) 
# X, Y = transform.drop(patient,startXIndex=0,YIndex=2,dropColunm=['肿瘤1，结核4，慢性炎症2，急性炎症3，漏出液0'])    
# X, Y = transform.drop(patient,startXIndex=0,YIndex=3,dropColunm=['恶性1，良性0'])    
X, Y = transform.drop(patient,startXIndex=0,YIndex=-1,dropColunm=['Y'])    

for name in X.columns:
    X[name] = X[name].apply(str2Nan)
    X[name] = X[name].fillna(X[name].median())

In [ ]:
'''
kfold
'''
k = 10
X = transform.normalization(X)
result_svm, result_gbdt,result_tree,result_randomtree,result_lg,result_knn,result_lgb = [],[], [], [],[],[],[]
false_index = []
test_x, test_y = X[-200:],Y[-200:]
X, Y = X[:-200],Y[:-200]


In [ ]:
ModelDict = {
    'SVM':svm.SVC(random_state=seed, probability=True),
    'DT':DecisionTreeClassifier(max_depth=10,
                                          random_state=seed),
    'GBDT':GradientBoostingClassifier(random_state=seed),
    'RF':RandomForestClassifier(random_state=seed,
                                          min_samples_leaf=10),
    'KNN':KNeighborsClassifier(),
    'LR':LogisticRegression(random_state=400,max_iter=100),
    'XGB':XGBClassifier(),
    'ET':ExtraTreesClassifier(n_estimators=5, random_state=42),
    'Ada':AdaBoostClassifier(n_estimators=50, random_state=42),
    'LightGBM':lgb.LGBMClassifier(num_leaves=400,random_state=42,verbosity=-1),
    'DNN':dnn}  
for i in range(k):
    if i!=4:
        continue
    print('第{:d}代'.format(i+1))
    X_train, y_train, x_valid, y_valid,Y_index = get_k_fold_data(k,i,X,Y,transform.confuse_index)
    pre = PredictWay(X_train, y_train, x_valid, y_valid,seed,Y_index)
    for model_name, model in ModelDict.items():
        matric = pre.model_train(model_name, model)
        if model_name=='LightGBM':
            model_shap = pre.model_fit
    pre.show_roc()


In [ ]:
result = {
    'LightGBM':[]
}

'''
获取特征重要性
'''
temp = model_shap.feature_importances_[:]
print(temp)
importance = []
for i in range(len(temp)):
    importance.append(temp.argmax())
    temp[temp.argmax()]=-1
importance.reverse()

In [ ]:
X[X.columns[importance[-10:]]]

In [ ]:
'''
绘制kfold的roc
'''
from sklearn.metrics import roc_curve, auc
roc_lgbs = []
X_in = X.drop(X.columns[importance[:12]],axis=1)  
X_test_drop = test_x.drop(X.columns[importance[:12]],axis=1)
k = 5
false_index = []
for i in range(k):
    print('第{:d}代'.format(i+1))
    X_train, y_train, x_valid, y_valid,Y_index = get_k_fold_data(k,i,X_in,Y,transform.confuse_index)
    pre = PredictWay(X_train, y_train, x_valid, y_valid,seed,Y_index)
    pre.show = False
    pre.model_train('LightGBM',ModelDict['LightGBM'])
    f,t = pre.dict_auc['LightGBM']
    roc_lgbs.append([f,t])
    result = np.array(pre.model_fit.predict_proba(X_test_drop))[:, 1]
    f, t, th = roc_curve(test_y, result)
    print('外部验证集 最终模型auc：', auc(f,t)) 

plt.figure(figsize=(10,10))
tprs = []
mean_fpr = np.linspace(0,1,100)
for index, i in enumerate(roc_lgbs):
    plt.plot(i[0],i[1], alpha = 0.3, label=f'ROC fold {index}(AUC={auc(i[0],i[1]):.4f})')
    # sum0 += i[0]
    # sum1 += i[1]
    tprs.append(np.interp(mean_fpr, i[0], i[1]))
    tprs[-1][0] = 0.0
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr,mean_tpr,color='b',label=r'Mean ROC (area=%0.4f)'%mean_auc,lw=2,alpha=.8)
std_tpr = np.std(tprs, axis=0)
tprs_upper=np.minimum(mean_tpr+std_tpr,1)
tprs_lower=np.maximum(mean_tpr-std_tpr,0)
print(mean_fpr, tprs_lower, tprs_upper)
plt.fill_between(mean_fpr,tprs_lower,tprs_upper,color='gray',alpha=.2,label='± 1 SD')
plt.plot([0,1],[0,1],linestyle='--',lw=2,color='r',label='Chance')
plt.xlabel('False Positive Rate(1-specificity)')
plt.ylabel('True Positive Rate(Sensitivity)')
plt.title('Five-fold')
plt.legend(loc='lower right')
plt.show()   


In [ ]:
'''
外部验证 10features
'''
from sklearn.metrics import roc_curve, auc
f, t, th = roc_curve(test_y, result)
print('auc:', auc(f,t))

In [ ]:

print([importance])
result = {}
for index_i in range(len(importance)):
    X_drop = X.drop(X.columns[importance[:index_i]],axis=1)  
    k = 5
    for i in range(k):
        if i!=4:
            continue
        X_train, y_train, x_valid, y_valid,Y_index = get_k_fold_data(k,i,X_drop,Y,transform.confuse_index)
        pre = PredictWay(X_train, y_train, x_valid, y_valid,seed,Y_index)
        for model_name, model in ModelDict.items():
            matric = pre.model_train(model_name, model)
    for model_name, list_f_t in pre.dict_auc.items():
        result.setdefault(model_name,[]).append(auc(list_f_t[0], list_f_t[1]))
    print(result)


result['GBDT'].reverse()
result['LightGBM'].reverse()
result['SVM'].reverse()
result['DT'].reverse()
result['RF'].reverse()
# result['LR'].reverse()
result['KNN'].reverse()
result['ET'].reverse()
result['XGB'].reverse()
result['DNN'].reverse()
result['Ada'].reverse()
plt.figure(figsize=(10,10))
plt.xticks(range(2, 19, 1), labels=[str(i) for i in range(2, 19, 1)])
indice = np.arange(3, 19, 1)
plt.xlabel('Numbers of features')
plt.ylabel('Area under the ROC')
plt.plot(indice,result['GBDT'][2:19:1],':',label='GBDT')
plt.plot(indice,result['LightGBM'][2:19:1],'--',label='LightGBM')
plt.plot(indice,result['SVM'][2:19:1],'-.',label='SVM')
plt.plot(indice,result['DT'][2:19:1],'-',label='DT')
plt.plot(indice,result['RF'][2:19:1],'-.',label='RF')
plt.plot(indice,result['ET'][2:19:1],'-.',label='ET')
plt.plot(indice,result['Ada'][2:19:1],':',label='AdaBoost')
plt.plot(indice,result['DNN'][2:19:1],':',label='DNN')
plt.plot(indice,result['XGB'][2:19:1],':',label='XGB')
plt.plot(indice,result['KNN'][2:19:1],'--',label='KNN')
plt.scatter(indice,result['GBDT'][2:19:1],s=20)
plt.scatter(indice,result['LightGBM'][2:19:1],s=20)
plt.scatter(indice,result['SVM'][2:19:1],s=20)
plt.scatter(indice,result['DT'][2:19:1],s=20)
plt.scatter(indice,result['RF'][2:19:1],s=20)
plt.scatter(indice,result['ET'][2:19:1],s=20)
plt.scatter(indice,result['XGB'][2:19:1],s=20)
plt.scatter(indice,result['Ada'][2:19:1],s=20)
plt.scatter(indice,result['DNN'][2:19:1],s=20)
plt.scatter(indice,result['KNN'][2:19:1],s=20)

plt.axvline(11,c='black',ls='-.')
plt.legend()
plt.show()

In [ ]:
result_metric = {
    'AUC':[],
    'Sensitivity':[],
    'Specificity':[],
    'F1':[],
    'PPV':[],
    'NPV':[],
    'Acc':[]
}
from scipy.stats import ttest_ind
result_imp = []
for index_i in range(len(importance)):
    X_in = X.drop(X.columns[importance[:index_i]],axis=1)  
    Xtest_in = test_x.drop(test_x.columns[importance[:index_i]],axis=1)  
    k = 5
    result_svm, result_gbdt,result_tree,result_randomtree,result_lg,result_knn,result_lgb = [],[], [], [],[],[],[]
    false_index = []
    for i in range(k):
        if i!=4:
            continue
        X_train, y_train, x_valid, y_valid,Y_index = get_k_fold_data(k,i,X_in,Y,transform.confuse_index)
        pre = PredictWay(X_train, y_train, x_valid, y_valid,seed,Y_index)
        sensitive,specificity,ppv,npv,acc,f1 = pre.model_train('LightGBM',lgb.LGBMClassifier(num_leaves=400,random_state=42,verbosity=-1))
        #  'LightGBM':lgb.LGBMClassifier(num_leaves=400,random_state=42,verbosity=-1)
        f,t = pre.dict_auc['LightGBM'][0], pre.dict_auc['LightGBM'][1]
        if X_train.shape[1]==18:
            result_imp.append([f,t,22,auc(f,t),None,None])
            auc0 = auc(f,t)
            result0 = pre.result
        if X_train.shape[1] in [11,10,9,3]:
            result_imp.append([f,t,X_train.shape[1],auc(f,t),auc0-auc(f,t),ttest_ind(auc(f,t), auc0)])
        # if X_train.shape[1]==10:
        #     result_imp.append([f,t,10,auc(f,t),0.0029,0.4131])
        # if X_train.shape[1]==9:
        #     result_imp.append([f,t,9,auc(f,t),0.0145,0.2090])
        # if X_train.shape[1]==3:
        #     result_imp.append([f,t,3,auc(f,t),0.1184,0.00015])
    result_metric['AUC'].append(auc(f,t))
    result_metric['Sensitivity'].append(sensitive)
    result_metric['Specificity'].append(specificity)
    result_metric['F1'].append(f1)
    result_metric['PPV'].append(ppv)
    result_metric['NPV'].append(npv)
    result_metric['Acc'].append(acc)
'''
画图
'''
result_metric['AUC'].reverse()
result_metric['Sensitivity'].reverse()
result_metric['Specificity'].reverse()
result_metric['F1'].reverse()
result_metric['NPV'].reverse()
result_metric['PPV'].reverse()
result_metric['Acc'].reverse()
plt.figure(figsize=(10,8))
plt.xticks(range(1,19,1), labels=[str(i) for i in range(1,19,1)])

print(result_metric['AUC'])
print(result_metric['Sensitivity'])
print(result_metric['Specificity'])
print(result_metric['F1'])
print(result_metric['NPV'])
print(result_metric['PPV'])
print(result_metric['Acc'])


indice = np.arange(3,19,1)
plt.xlabel('Numbers of features')
plt.ylabel('Area under the ROC')
plt.plot(indice,result_metric['AUC'][2:18:1],':',label='AUC')
plt.plot(indice,result_metric['Sensitivity'][2:18:1],'--',label='Sensitivity')
plt.plot(indice,result_metric['Specificity'][2:18:1],'-.',label='Specificity')
plt.plot(indice,result_metric['F1'][2:18:1],'-',label='F1')
plt.scatter(indice,result_metric['AUC'][2:18:1],s=20)
plt.scatter(indice,result_metric['Sensitivity'][2:18:1],s=20)
plt.scatter(indice,result_metric['Specificity'][2:18:1],s=20)
plt.scatter(indice,result_metric['F1'][2:18:1],s=20)
plt.axvline(11,c='black',ls='-.')
plt.legend()
plt.show()

In [ ]:
pd.DataFrame(result_metric)